In [1]:
import time

import torch
import copy
import torchvision.models as models
from torchvision import datasets, models, transforms

import numpy as np

import cv2

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

In [2]:
pretrained_shuffle_net = models.shufflenet_v2_x1_0( progress=True)

In [3]:
BATCH_SIZE = 100

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.ToTensor()
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.ToTensor()
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.ToTensor()
    ])
}
transform = transforms.Compose([transforms.Resize(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomRotation(30),
                                transforms.ToTensor()])
device = torch.device("cpu")

In [5]:
def load_data(folder):
    dataset = datasets.ImageFolder(folder, transform)
    data_num = len(dataset)
    
    train_size = int(data_num * 0.7)
    valid_size = int(data_num * 0.2)
    test_size = data_num - train_size - valid_size
    train_data, valid_data, test_data = torch.utils.data.random_split(dataset, (train_size, valid_size, test_size))
    
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)
    
    image_datasets = {"train": train_data, "val": valid_data, "test": test_data}
    dataloaders = {"train": train_loader, "val": valid_loader, "test": test_loader}
    dataset_sizes = {"train": train_size, "val": valid_size, "test": test_size}
    return image_datasets, dataloaders, dataset_sizes

In [6]:
image_datasets, dataloaders, dataset_sizes = load_data('CK+48')

In [7]:
# for param in pretrained_shuffle_net.parameters():
#     param.requires_grad = True

num_ftrs = pretrained_shuffle_net.fc.in_features
pretrained_shuffle_net.fc = torch.nn.Linear(num_ftrs, 10)

pretrained_shuffle_net = pretrained_shuffle_net.to(device)

criterion = torch.nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = torch.optim.SGD(pretrained_shuffle_net.parameters(), lr=0.01, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=15, gamma=0.1)

In [8]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [9]:
best_model = train_model(pretrained_shuffle_net, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------


C:\Users\wusix\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


KeyboardInterrupt: 

In [ ]:
def test_model(model, criterion, optimizer, scheduler):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

#     print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    # Each epoch has a training and validation phase
#         for phase in ['train', 'val']:
#             if phase == 'train':
#                 scheduler.step()
#                 model.train()  # Set model to training mode
#             else:
    model.eval()   # Set model to evaluate mode

    running_loss = 0.0
    running_corrects = 0

    # Iterate over data.
    for inputs, labels in dataloaders["test"]:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        # track history if only in train
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # backward + optimize only if in training phase
#                     if phase == 'train':
#                         loss.backward()
#                         optimizer.step()

        # statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / dataset_sizes["test"]
    epoch_acc = running_corrects.double() / dataset_sizes["test"]

    print('{} Loss: {:.4f} Acc: {:.4f}'.format(
        "test", epoch_loss, epoch_acc))

    # deep copy the model
#             if phase == 'val' and epoch_acc > best_acc:
#                 best_acc = epoch_acc
#                 best_model_wts = copy.deepcopy(model.state_dict())

    print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

In [ ]:
test_model(best_model, criterion, optimizer_ft, exp_lr_scheduler)

In [ ]:
data_dir = 'CK+48'
dsets = datasets.ImageFolder(data_dir)
dset_loaders = torch.utils.data.DataLoader(dsets, batch_size=4,
                                               shuffle=True, num_workers=4)
dset_classes = dsets.classes
len(dsets)
dsets[900][0]

In [ ]:
data_num = len(dsets)
train_size = int(0.7*data_num)
train, valid = torch.utils.data.random_split(dsets, (train_size, data_num-train_size))

In [ ]:
pretrained_shuffle_net

In [ ]:
image = cv2.imread('./CK+48/anger/S010_004_00000017.png')
resized_image = cv2.resize(image, (224, 224))
plt.imshow(resized_image, cmap='gray')
plt.show()

In [ ]:
image

In [ ]:
0.97/(0.03*9999 + 0.97)

In [ ]:
0.144*(0.1*0.096+0.6*0.224)/(0.08*(0.6*0.096+0.1*0.224)+0.096*(0.3*0.096+0.3*0.224)+0.144*(0.1*0.096+0.6*0.224))